# Composite AutoML

In [4]:
import pandas as pd

df = pd.read_csv('tmp/extracted_data_cleaned_1.csv')
X = df.iloc[:,:-1]
X = X.loc[:, ~X.columns.str.contains('^Unnamed')]
y = df.iloc[:,-1]

df.head()

,Unnamed: 0,"Flow rate__augmented_dickey_fuller__attr_""teststat""","Flow rate__augmented_dickey_fuller__attr_""usedlag""",Flow rate__cid_ce__normalize_True,Flow rate__count_above_mean,Flow rate__count_below_mean,"Flow rate__fft_aggregated__aggtype_""kurtosis""","Flow rate__fft_aggregated__aggtype_""skew""","Flow rate__fft_coefficient__coeff_10__attr_""angle""","Flow rate__fft_coefficient__coeff_11__attr_""angle""",...,Zone9_Pressure__quantile__q_0.7,Zone9_Pressure__range_count__max_1__min_-1,Zone9_Pressure__ratio_beyond_r_sigma__r_0.5,Zone9_Pressure__spkt_welch_density__coeff_2,Zone9_Pressure__spkt_welch_density__coeff_5,Zone9_Pressure__spkt_welch_density__coeff_8,Zone9_Pressure__sum_of_reoccurring_data_points,Zone9_Pressure__symmetry_looking__r_0.1,Zone9_Pressure__symmetry_looking__r_0.15000000000000002,target
0,0,-0.917134,3.0,2.027651,439.0,418.0,-5.207888,3.018917,128.443307,-134.379691,...,0.0,691.0,0.116686,2.586976e+07,2.702030e+05,4.303616e+04,0.0,0.0,1.0,1
1,1,-0.829793,5.0,2.095977,439.0,422.0,-5.110457,3.015963,118.192733,-127.783891,...,0.0,705.0,0.106852,8.413668e+06,4.673739e+05,4.960634e+04,0.0,1.0,1.0,0
2,2,-1.476037,1.0,2.915139,414.0,478.0,-5.713558,2.575471,-131.016670,-130.032831,...,0.0,700.0,0.125561,1.012102e+07,3.076063e+05,3.522584e+04,0.0,1.0,1.0,1
3,3,-0.961903,2.0,2.037617,491.0,412.0,-5.395782,2.942215,-90.795881,26.986556,...,0.0,692.0,0.156146,4.862382e+09,7.062084e+07,1.368867e+07,0.0,0.0,1.0,0
4,4,-1.351336,4.0,2.862844,528.0,411.0,-5.909401,2.586751,37.502317,-160.207602,...,0.0,747.0,0.123536,6.137353e+07,5.278134e+05,9.860641e+05,0.0,1.0,1.0,0


### TPOT

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

# generations: int, optional (default=100)
#    Number of iterations to the run pipeline optimization process. Must be a positive number.
#    Generally, TPOT will work better when you give it more generations (and therefore time) to optimize the pipeline.
#    TPOT will evaluate population_size + generations × offspring_size pipelines in total. 
tpot = TPOTClassifier(generations=100, population_size=50, verbosity=2, n_jobs=8)
tpot.fit(X_train, y_train)

tpot.score(X_test, y_test)

0.6